### <u>**COMP-548DL**</u>
### <u>**Big Data Management and Processing**</u>
### <u>**Semester Project**</u>

###

#### <u>**Part 1:**</u> 
#### This notebook contains the code used for uploading the data on Firestore in document format.
#### A few queries are performed for experimentation, but then we transform the data into a JSON file in order to continue with the main approach.
#### (Utilizing PySpark, uploading the data on GCS and provisioning a Dataproc cluster for the rest of the analysis)

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
from collections import defaultdict
import pandas as pd
import time
import json

In [12]:
 %env GOOGLE_APPLICATION_CREDENTIALS=C:\Users\ska_p\OneDrive\Desktop\MSc Data Science\Fall Semester 2024-25\Big Data Management and Processing\Final Project\big-data-final-project-445611-3749e103c9e3.json

env: GOOGLE_APPLICATION_CREDENTIALS=C:\Users\ska_p\OneDrive\Desktop\MSc Data Science\Fall Semester 2024-25\Big Data Management and Processing\Final Project\big-data-final-project-445611-3749e103c9e3.json


In [9]:
# Initializing Google Firestore
cred = credentials.Certificate("C:/Users/ska_p/OneDrive/Desktop/MSc Data Science/Fall Semester 2024-25/Big Data Management and Processing/Final Project/big-data-final-project-445611-3749e103c9e3.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

# Creating a Firestore collection name
collection_name = "crime_data"


In [3]:
# Loading our dataset 
file_path = "london_crime.csv"  
data = pd.read_csv(file_path)

In [14]:
# This is a function for uploading the data in batches
# Calling the function separately will help us tackle any issues in a timely and clean manner
def upload_to_firestore(batch_size=499):
    total_rows = len(data)
    print(f"Uploading {total_rows} rows to Firestore...")

    for start_idx in range(0, total_rows, batch_size):
        batch = db.batch()
        end_idx = min(start_idx + batch_size, total_rows)
        print(f"Uploading rows {start_idx} to {end_idx}...")
        
        # Prepare documents for the batch
        for _, row in data.iloc[start_idx:end_idx].iterrows():
            doc_ref = db.collection(collection_name).document()
            doc_data = {
                "lsoa_code": row["lsoa_code"],
                "borough": row["borough"],
                "major_category": row["major_category"],
                "minor_category": row["minor_category"],
                "value": int(row["value"]),
                "year": int(row["year"]),
                "month": int(row["month"]),
            }
            batch.set(doc_ref, doc_data)

        # Commit the batch
        batch.commit()

    # Just a little reassurance...!    
    print("Data upload complete!")


#### Now we call the function to finish the upload. It is a very time-consuming process, but it obviously has to be executed only once.

In [13]:
# Call the function
upload_to_firestore()

Uploading 13490604 rows to Firestore...
Uploading rows 0 to 499...
Uploading rows 499 to 998...
Uploading rows 13489966 to 13490465...
Uploading rows 13490465 to 13490604...
Data upload complete!


#

#### Let's perform a few queries to test the connection and that everything has been uploaded properly.
#### We will limit the query replies in order to avoid potential issues emerging from the vast data volume on Firestore.

In [10]:
# Test Firestore connection by fetching the first document of the collection
# We can verify the accuracy of the answer by simultaneously checking the firestore documents from the cloud
collection_name = "crime_data"
query = db.collection(collection_name).limit(1).stream()

for doc in query:
    print(f"Document ID: {doc.id}, Data: {doc.to_dict()}")


Document ID: 0000pqVKxcVDaXRNeOap, Data: {'major_category': 'Criminal Damage', 'year': 2009, 'value': 0, 'lsoa_code': 'E01001111', 'borough': 'Croydon', 'month': 3, 'minor_category': 'Criminal Damage To Other Building'}


In [11]:
# Count total documents in the collection
# We limit this query to the first 100.000 documents to avoid computational power issues
collection_name = "crime_data"
docs = db.collection(collection_name).limit(100000).stream()
doc_count = sum(1 for _ in docs)
print(f"Total documents in '{collection_name}': {doc_count}")


Total documents in 'crime_data': 100000


In [13]:
# Query for a specific borough
# Again this is limited to the first 10 entries for simplicity reasons
borough = "Croydon"
query = db.collection(collection_name).where("borough", "==", borough).limit(10).stream()

print(f"Crime records for borough '{borough}':")
for doc in query:
    print(f"Document ID: {doc.id}, Data: {doc.to_dict()}")


Crime records for borough 'Croydon':
Document ID: 0000pqVKxcVDaXRNeOap, Data: {'major_category': 'Criminal Damage', 'year': 2009, 'value': 0, 'lsoa_code': 'E01001111', 'borough': 'Croydon', 'month': 3, 'minor_category': 'Criminal Damage To Other Building'}
Document ID: 000AWa1pptq1ZvATBsPM, Data: {'major_category': 'Theft and Handling', 'year': 2015, 'value': 0, 'lsoa_code': 'E01001138', 'borough': 'Croydon', 'month': 11, 'minor_category': 'Handling Stolen Goods'}
Document ID: 000DZCX3lSJXiYy4hyhA, Data: {'major_category': 'Robbery', 'year': 2015, 'value': 0, 'lsoa_code': 'E01000997', 'borough': 'Croydon', 'month': 7, 'minor_category': 'Personal Property'}
Document ID: 000sSzwjQB2BuTKS1wDR, Data: {'major_category': 'Theft and Handling', 'year': 2010, 'value': 2, 'lsoa_code': 'E01001125', 'borough': 'Croydon', 'month': 8, 'minor_category': 'Theft From Shops'}
Document ID: 0016uKFb4oAPegGKv79O, Data: {'major_category': 'Criminal Damage', 'year': 2016, 'value': 0, 'lsoa_code': 'E01000995'

#

#### Queries in Firestore are working fine, but we will use a different approach to continue from now on.
#### To end this notebook, we transform the dataset to JSON format, for uploading to a Dataproc bucket and continuing from there.

In [2]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("CSVtoJSON") \
    .master("local[*]") \
    .getOrCreate()


In [3]:
# Path to our CSV file
csv_file_path = "london_crime.csv"  

# Load the CSV file
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+---------+--------------------+--------------------+--------------------+-----+----+-----+
|lsoa_code|             borough|      major_category|      minor_category|value|year|month|
+---------+--------------------+--------------------+--------------------+-----+----+-----+
|E01001116|             Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646|           Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|             Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774|           Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|          Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
|E01001320|              Ealing|  Theft and Handling|         Other Theft|    0|2012|    5|
|E01001342|              Ealing|Violence Against ...|    Offensive Weapon|    0|2010|    7|
|E01002633|            Hounslow|             Robbery|   Personal Property|    0|

In [4]:
# Path to save the JSON file
json_file_path = "london_crime_json.json" 

# Save as JSON
df.write.json(json_file_path)

print(f"JSON file saved at {json_file_path}")


JSON file saved at london_crime_json.json
